Load Julia packages (libraries)

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "08")
cd(ProjDir)

### snippet 5.1

In [3]:
d = CSV.read(rel_path("..", "data", "rugged.csv"), delim=';');
df = convert(DataFrame, d);

dcc = filter(row -> !(ismissing(row[:rgdppc_2000])), df)
dcc[:log_gdp] = log.(dcc[:rgdppc_2000])
dcc[:cont_africa] = Array{Float64}(convert(Array{Int}, dcc[:cont_africa]))
dcc[:rugged] = convert(Array{Float64}, dcc[:rugged])
first(dcc[[:rugged, :cont_africa, :log_gdp]], 5)

,rugged,cont_africa,log_gdp
,Float64,Float64,Float64
1,0.858,1.0,7.49261
2,3.427,0.0,8.21693
3,0.769,0.0,9.93326
4,0.775,0.0,9.40703
5,2.688,0.0,7.79234


Define the Stan language model

In [4]:
m_8_1 = "
data{
    int N;
    vector[N] log_gdp;
    vector[N] cont_africa;
    vector[N] rugged;
    vector[N] rugged_cont_africa;
}
parameters{
    real a;
    real bR;
    real bA;
    real bAR;
    real sigma;
}
model{
    vector[N] mu = a + bR * rugged + bA * cont_africa + bAR * rugged_cont_africa;
    sigma ~ uniform( 0 , 10 );
    bAR ~ normal( 0 , 10 );
    bA ~ normal( 0 , 10 );
    bR ~ normal( 0 , 10 );
    a ~ normal( 0 , 100 );
    log_gdp ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [5]:
stanmodel = Stanmodel(name="m_8_1",
monitors = ["a", "bR", "bA", "bAR", "sigma"],
model=m_8_1, output_format=:mcmcchains);


File /Users/rob/.julia/dev/StanModels/scripts/08/tmp/m_8_1.stan will be updated.



Input data for cmdstan

In [6]:
m_8_1_data = Dict("N" => size(dcc, 1),
"log_gdp" => dcc[:log_gdp],  "rugged" => dcc[:rugged],
"cont_africa" => dcc[:cont_africa],
"rugged_cont_africa" => dcc[:rugged] .* dcc[:cont_africa] );

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_8_1_data, ProjDir, diagnostics=false,
  summary=true, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
       mean   sd  5.5% 94.5% n_eff Rhat
a      9.22 0.14  9.00  9.46   282    1
bR    -0.21 0.08 -0.33 -0.08   275    1
bA    -1.94 0.24 -2.33 -1.59   268    1
bAR    0.40 0.14  0.18  0.62   271    1
sigma  0.96 0.05  0.87  1.04   339    1
"

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -0.882234, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/scripts/08/tmp/m_8_1.stan' at line 22)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -0.536416, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/scripts/08/tmp/m_8_1.stan' at line 22)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecifie

"\n       mean   sd  5.5% 94.5% n_eff Rhat\na      9.22 0.14  9.00  9.46   282    1\nbR    -0.21 0.08 -0.33 -0.08   275    1\nbA    -1.94 0.24 -2.33 -1.59   268    1\nbAR    0.40 0.14  0.18  0.62   271    1\nsigma  0.96 0.05  0.87  1.04   339    1\n"

Describe the draws

In [8]:
describe(chn)

2-element Array{ChainDataFrame,1}

Summary Statistics
. Omitted printing of 1 columns
│ Row │ parameters │ mean      │ std       │ naive_se    │ mcse        │
│     │ Symbol     │ Float64   │ Float64   │ Float64     │ Float64     │
├─────┼────────────┼───────────┼───────────┼─────────────┼─────────────┤
│ 1   │ a          │ 9.22422   │ 0.145874  │ 0.00230647  │ 0.00339064  │
│ 2   │ bA         │ -1.94943  │ 0.232188  │ 0.00367122  │ 0.00514314  │
│ 3   │ bAR        │ 0.394698  │ 0.135929  │ 0.00214923  │ 0.00273548  │
│ 4   │ bR         │ -0.203487 │ 0.0804628 │ 0.00127223  │ 0.00183684  │
│ 5   │ sigma      │ 0.951758  │ 0.0533075 │ 0.000842865 │ 0.000785924 │

Quantiles
. Omitted printing of 1 columns
│ Row │ parameters │ 2.5%      │ 25.0%     │ 50.0%     │ 75.0%     │
│     │ Symbol     │ Float64   │ Float64   │ Float64   │ Float64   │
├─────┼────────────┼───────────┼───────────┼───────────┼───────────┤
│ 1   │ a          │ 8.93499   │ 9.12764   │ 9.22541   │ 9.32018   │
│ 2   │ bA 

End of `m8.1s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*